In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from time import sleep
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy


In [13]:

class Game:
    def __init__(self):
        self.place = np.array([0, 0])
    def tell_state(self):
        return self.place

    def move(self, x):
        place = self.place.copy()
        if np.array_equal(place, np.array([0, 0])):
            if x == 1:
                self.place = np.array([0, 1])
                print(f'move right and now is in : {self.place} and you earn the prize:{-1}')
                return -1
            else:
                self.place = np.array([1, 0])
                print(f'move left and now is in : {self.place} and you earn the prize:{-1}')
                return -1

        elif np.array_equal(place, np.array([0, 1])):
            if x == 1:
                self.place = np.array([1, 1])
                print(f'move right and now is in : {self.place} and you earn the prize:{10}')
                return 10
            else:
                self.place = np.array([0, 0])
                print(f'move left and now is in : {self.place} and you earn the prize:{-10}')
                return -10

        elif np.array_equal(place, np.array([1, 0])):
            if x == 1:
                self.place = np.array([0, 0])
                print(f'move right and now is in : {self.place} and you earn the prize:{-10}')
                return -10
            else:
                self.place = np.array([1, 1])
                print(f'move left and now is in : {self.place} and you earn the prize:{10}')
                return 10

        elif np.array_equal(place, np.array([1, 1])):
            if x == 1:
                self.place = np.array([1, 0])
                print(f'move right and now is in : {self.place} and you earn the prize:{-1}')
                return -1
            else:
                self.place = np.array([0, 1])
                print(f'move left and now is in : {self.place} and you earn the prize:{-1}')
                return -1


In [3]:
#game with person
game = Game()
finished_game = False
while not finished_game :
  a = int(input())
  if a ==2 :
    finished_game = True
  else  :
    prize = game.move(a)
    if prize == 10  :
      print('you win the game')
      finished_game = True




1
move right and now is in : [0 1] and you earn the prize:-1
1
move right and now is in : [1 1] and you earn the prize:10
you win the game


In [4]:
#game randomly
game = Game()
finished_game = False
while not finished_game :
  a = np.random.choice([0 , 1 ])
  sleep(1)
  prize = game.move(a)
  if prize == 10  :
    print('you win the game')
    finished_game = True



move left and now is in : [1 0] and you earn the prize:-1
move left and now is in : [1 1] and you earn the prize:10
you win the game


In [22]:
class Ai_player(tf.keras.Model)  :
  def __init__(self):
    super().__init__()
    self.layer1 = Dense(10 , activation='relu')
    self.layer2 = Dense(5 , activation='relu')
    self.layer3 = Dense(2 , activation='softmax')
    self.optimizer= Adam(learning_rate=0.01)
  def call(self , x ) :
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    return x
  def train(self , rewards , states , actions)  :
      with tf.GradientTape() as tape :
          log_probs = []

          for state, action in zip(states, actions):
              state = np.array(state).reshape(1, -1)
              action_prob = self(state)
              log_prob = tf.math.log(action_prob[0][action])
              log_probs.append(log_prob)

          R = 0
          total_reward = 0
          for log_prob, reward in zip(log_probs[::-1], rewards[::-1]):
              R = reward + 0.99 * R  # تخفیف پاداش‌ها
              total_reward += -log_prob * R
      grad = tape.gradient(total_reward , self.trainable_variables)
      self.optimizer.apply_gradients(zip(grad , self.trainable_variables))

In [32]:
#now try to learn the rules:
#I want to play one hundred times :
player = Ai_player()
game_ai = Game()
epsilon = 0
for i in range(100) :
    game_done = False
    rewards = []
    actions = []
    states = []
    while not game_done :
        state = game_ai.tell_state()
        states.append(state)
        state = tf.convert_to_tensor(state, dtype=tf.float32)
        state = tf.reshape(state, (1, -1))  # اضافه کردن بعد batch
        prob = player(state)
        if np.random.rand(1) < epsilon  :
            action = np.argmax(prob)
        else :
            action = np.random.choice([0 , 1])
        reward = game_ai.move(action)
        rewards.append(reward)
        actions.append(action)

        if reward == 10 :
          game_done= True
    player.train(rewards , states , actions)
    print(f'game {i} finished')
    #sleep(0.1)
    epsilon +=0.1





move left and now is in : [1 0] and you earn the prize:-1
move right and now is in : [0 0] and you earn the prize:-10
move right and now is in : [0 1] and you earn the prize:-1
move right and now is in : [1 1] and you earn the prize:10
game 0 finished
move right and now is in : [1 0] and you earn the prize:-1
move left and now is in : [1 1] and you earn the prize:10
game 1 finished
move right and now is in : [1 0] and you earn the prize:-1
move left and now is in : [1 1] and you earn the prize:10
game 2 finished
move right and now is in : [1 0] and you earn the prize:-1
move left and now is in : [1 1] and you earn the prize:10
game 3 finished
move right and now is in : [1 0] and you earn the prize:-1
move left and now is in : [1 1] and you earn the prize:10
game 4 finished
move right and now is in : [1 0] and you earn the prize:-1
move left and now is in : [1 1] and you earn the prize:10
game 5 finished
move right and now is in : [1 0] and you earn the prize:-1
move left and now is in 

In [35]:
#one time game with ai :

game = Game()
finished_game = False
while not finished_game :
  state=game.tell_state()
  state = tf.convert_to_tensor(state, dtype=tf.float32)
  state = tf.reshape(state, (1, -1))
  prob = player(state)
  action = np.argmax(prob)
  print(f'and the probablity is the {prob[0][action]} ')
  prize = game.move(action)
  sleep(1)
  if prize == 10 :
    print('you win the game')
    finished_game = True

and the probablity is the 0.8038496375083923 
move left and now is in : [1 0] and you earn the prize:-1
and the probablity is the 0.9975413084030151 
move left and now is in : [1 1] and you earn the prize:10
you win the game
